In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx

In [2]:
text='''
Ajinkya Rahane's batting masterclass followed by Ravichandran Ashwin's bowling heroics sealed Team India's massive 337-run win over South Africa in the fourth and final Test of the bilateral series at Delhi in 2015. Local hero Virat Kohli, who was at the helm, had described Team India's famous win as the most special of the bilateral series between the two nations.

Just like how India humbled Australia in the Border-Gavaskar Trophy 2023, Kohli and Co. wrapped up the Test match against South Africa within three days at the time. In the thrilling contest at Arun Jaitely Stadium, Kohli's men had to grind hard for each wicket in the 2nd innings of the Test match. Batting icons AB de Villiers and Hasim Amla had epitomised South Africa's resistance at Delhi.

For former Indian fielding coach R Sridhar, the Delhi Test match between India and South Africa served him with one of the toughest lessons of his coaching career. Taking a trip down memory lane in his recently-published book 'Coaching Beyond: My Days with the Indian Cricket Team', the former Indian fielding coach revealed that he wanted to walk away from his job after the 2015 Test match in Delhi.

“I learnt some harsh lessons reasonably early on in my stint with the Indian team. One of the most impactful ones came at the Kotla, in the final Test of the 2015 series against South Africa. We already enjoyed a winning 2-0 lead, and by tea on day five, it seemed as if South Africa might succeed in preventing us from scoring a third successive victory. With AB putting on a stonewalling act for the ages and receiving support from Hashim Amla and Faf du Plessis on a dead track, South Africa were only five down at the final interval of the series. We were going to give it one final push on a gloomy December evening, but we were also reconciled to a stalemate, if it came to that,” Sridhar recalled.
'''

In [3]:
nltk.download('punkt')
sentences=sent_tokenize(text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
nltk.download('stopwords')

sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')
sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
w2v = Word2Vec(sentence_tokens, vector_size=10, min_count=1, epochs=1000)
sentence_embeddings = [[w2v.wv[word][0] for word in words] for words in sentence_tokens]
max_len = max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings = [np.pad(embedding, (0, max_len - len(embedding)), 'constant') for embedding in sentence_embeddings]

In [8]:
similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)

In [21]:
!pip install 'networkx<2.7'
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph)

In [22]:
top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])

In [23]:
for sent in sentences:
    if sent in top.keys():
        print(sent)

Local hero Virat Kohli, who was at the helm, had described Team India's famous win as the most special of the bilateral series between the two nations.
In the thrilling contest at Arun Jaitely Stadium, Kohli's men had to grind hard for each wicket in the 2nd innings of the Test match.
Batting icons AB de Villiers and Hasim Amla had epitomised South Africa's resistance at Delhi.
We already enjoyed a winning 2-0 lead, and by tea on day five, it seemed as if South Africa might succeed in preventing us from scoring a third successive victory.
